In [1]:
# This notebook loads Max's neural net 
# and gets eigenvectors of the graph laplacian

# import needed libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import scipy.linalg as linalg
import networkx as nx
import matplotlib.pyplot as plt

c:\Users\Xbrai\miniforge3\envs\ml\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\Xbrai\miniforge3\envs\ml\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\Xbrai\miniforge3\envs\ml\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\Xbrai\miniforge3\envs\ml\lib\site-packages\tensorflow\python\framework

In [ ]:
#load back the data
dir = "nn_training/"

#make a list where each entry is a matrix of weights or biases
params = []
for i in range(8):
    #for even i, type_string == "weights", for odd i type_string == "biases"
    type_string = 'weights' * (i%2 == 0) + 'biases' * (i%2 == 1)
    #params will have elements weights_0, biases_0, weights_1, biases_1, weights_2, biases_2, weights_3, biases_3
    params.append(np.load(dir +"nn_" + type_string + "_layer_" + str(int(i/2)) + ".npy", ))

In [ ]:
layer_sizes = np.array([784, 128, 64, 28, 10])
num_neurons = np.sum(layer_sizes)
#construct the adjacency matrix
A = np.zeros(shape = (num_neurons, num_neurons))

#each of the 784 neurons is adjacent to the 128
#each of the 128 neurons is adjacent to the 64 
#each of the 64 neurons is adjacent to the 128 and the 28
#each of the 28 neurons is adjacent to the 64 and the 10
#each of the 10 neurons is adjacent to the 28

offsets = np.r_[0, np.cumsum(layer_sizes)]   # e.g. [0, 784, 912, 976, 1004, 1014]
N = offsets[-1]                               # total neurons = 1014

# initialize adjacency
A = np.zeros((N, N), dtype=int)

# for each pair of consecutive layers i → i+1
for i in range(len(layer_sizes) - 1):
    start_i, end_i   = offsets[i],   offsets[i+1]
    start_j, end_j   = offsets[i+1], offsets[i+2]
    # connect all neurons in layer i to all in layer i+1
    A[start_i:end_i, start_j:end_j] = 1
    A[start_j:end_j, start_i:end_i] = 1  # mirror to make A symmetric
